In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, chi2


In [2]:
# Load data
data = pd.read_csv('complete_dataset.csv')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Separate features and labels
X = data.drop(['Label', 'Timestamp'], axis=1)
y = data['Label']
del data
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Create the RandomUnderSampler object
rus = RandomUnderSampler(random_state=42)
# Use it to resample the training data
X_train, y_train = rus.fit_resample(X_train, y_train)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
rfe = RFE(estimator=dt, n_features_to_select=10, step=20)

In [ ]:
for i in range(X.shape[1] - 9):
    # Fit RFE and train a decision tree with the selected features
    rfe.fit(X_train, y_train)
    selected_indices = rfe.get_support(indices=True)
    selected_features = X.columns[selected_indices]
    X_train_subset = X_train[:, selected_indices]
    X_test_subset = X_test[:, selected_indices]
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train_subset, y_train)

    # Make predictions and evaluate performance
    y_pred = dt.predict(X_test_subset)
    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # Print results and plot confusion matrix
    print("Iteration:", i+1)
    print("Selected features:", selected_features)
    print("Accuracy:", accuracy)
    print("Confusion matrix:\n", cm)
    print("----------------------------------------")
    plt.figure()
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('Confusion matrix (Iteration {})'.format(i+1))
    plt.show()

    # Remove the least important feature for the next iteration
    X_train = np.delete(X_train, rfe.ranking_.argmax(), axis=1)
    X_test = np.delete(X_test, rfe.ranking_.argmax(), axis=1)